In [1]:
# 1. Captures image and concatenates meme next to it
import cv2
import numpy as np
font = cv2.FONT_HERSHEY_SIMPLEX

def takeWebcam_picture(cap):
    ret,frame = cap.read() # return a single frame in variable `frame`
    cv2.putText(frame,'Carolina Obregon A01251983',(780, 50), font, 1,(255,255,255),2)
    return frame

# Taken from https://www.geeksforgeeks.org/concatenate-images-using-opencv-in-python/
# Used to images of different widths vertically
def vconcat_resize(img_list, interpolation  
                   = cv2.INTER_CUBIC): 
      # take minimum width 
    w_min = min(img.shape[1]  
                for img in img_list) 
      
    # resizing images 
    im_list_resize = [cv2.resize(img, 
                      (w_min, int(img.shape[0] * w_min / img.shape[1])), 
                                 interpolation = interpolation) 
                      for img in img_list] 
    # return final image 
    return cv2.vconcat(im_list_resize)

def show_picture(image, cap):
    while(True):
        cv2.imshow('img1',image) #display the captured image
        cv2.imwrite('guardar.png',image)
        if cv2.waitKey(1) & 0xFF == ord('y'): #save on pressing 'y' 
            cv2.destroyAllWindows()
        break
    cap.release()
    
cap = cv2.VideoCapture(1) # video capture source camera (Here webcam of laptop) 
webcam = takeWebcam_picture(cap)
meme = cv2.imread('meme.jpg',cv2.IMREAD_COLOR)
image = vconcat_resize([webcam, meme])
show_picture(image, cap)

In [ ]:
# 2. Filters color of webcam video of selected pixel
import cv2 as cv
import numpy as np

def getPixelBGR(x,y):
    colorsB = frame[y,x,0]
    colorsG = frame[y,x,1]
    colorsR = frame[y,x,2]
    return np.uint8([[[colorsB, colorsG, colorsR]]])

def getHSVBoundaries(hsv_img, currPixel):
    hsv_img = cv.cvtColor(currPixel, cv.COLOR_BGR2HSV)
    h, s, v = hsv_img[:, :, 0], hsv_img[:, :, 1], hsv_img[:, :, 2]
    h, s, v = h[0][0], s[0][0], v[0][0]
    lower_bound = np.array([h-10,100,100])
    upper_bound = np.array([h+10, 255,255])
    return [lower_bound, upper_bound]

def filterColor(hsv, boundaries):
    mask = cv.inRange(hsv, boundaries[0], boundaries[1])
    inverted = cv.bitwise_not(mask)
    return inverted

def showAndSaveResults(frame, inv):
    cv.imshow('og',frame)
    cv.imshow('inv',inv)
    cv.imwrite('inv.png',inv)
    cv.imwrite('og.png',frame)
    
def mouseRGB(event,x,y,flags,param):
    if event == cv.EVENT_LBUTTONDOWN: # checks mouse left button down condition
        currPixel = getPixelBGR(x,y) # captures BGR value of selected pixel
        boundaries = getHSVBoundaries(hsv_img, currPixel) # calculates hsv boundaries of selected pixel
        inverted = filterColor(hsv, boundaries) # filters color by creating mask and inverting it
        showAndSaveResults(frame, inverted) # saves final results

cv.namedWindow('mouseRGB')
cv.setMouseCallback('mouseRGB',mouseRGB)
cap = cv.VideoCapture(1)
while(1):
    # Take each frame
    _, frame = cap.read()
    # Convert BGR to HSV
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    cv.imshow('mouseRGB',frame)
    k = cv.waitKey(5) & 0xFF
    if k == 27:
        break
cv.destroyAllWindows()